Visualization of tremor catalog with bokeh

In [11]:
import pandas as pd

In [12]:
filename = '../data/tremors.pkl'
tremors = pd.read_pickle(filename)
tremors = tremors.sample(frac=0.1)

In [13]:
tremors.head()

,datetime,year,month,day,hour,minute,latitude,longitude,depth,errlat,errlon,errdepth,date,value,merc_long,merc_lat
6143,2010-03-17 04:35:00,2010,3,17,4,35,48.2140,-123.0220,44.00,4.792900,2.187940,9.899495,2010-03-17,0.479861,-1.369475e+07,6.142531e+06
19165,2010-08-17 16:35:00,2010,8,17,16,35,48.1625,-123.0750,44.50,1.287752,0.793795,3.703280,2010-08-17,0.917361,-1.370065e+07,6.133932e+06
8562,2010-03-21 09:06:00,2010,3,21,9,6,47.9580,-123.1620,23.40,1.210905,1.616167,2.880972,2010-03-21,0.157639,-1.371033e+07,6.099870e+06
9565,2010-06-02 17:50:00,2010,6,2,17,50,48.0975,-123.1175,20.75,1.887830,1.269712,2.217356,2010-06-02,0.054861,-1.370538e+07,6.123091e+06
15584,2010-08-14 18:49:00,2010,8,14,18,49,47.8760,-123.1280,17.20,4.323796,0.973532,2.774887,2010-08-14,0.585417,-1.370655e+07,6.086250e+06


In [15]:
from bokeh.plotting import figure, output_file, show
p = figure(title="plot", x_axis_label='lon', y_axis_label='lat')

# output to static HTML file
output_file("lines.html")

# add a line renderer with legend and line thickness
p.line(tremors.merc_long, tremors.merc_lat, legend="Temp.", line_width=2)

# show the results
show(p)


Import Python packages

In [1]:
from bokeh.io import curdoc
from bokeh.layouts import column, gridplot, layout, widgetbox
from bokeh.models import BoxZoomTool, ColumnDataSource, PanTool, ResetTool, WheelZoomTool
from bokeh.models.widgets import DateSlider
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON

from os.path import join, dirname
import pandas as pd

Define functions

In [2]:
def make_plot(selected_data_source, unselected_data_source):    
    tools = [ResetTool(), PanTool(), BoxZoomTool(), WheelZoomTool()]
    
    xmin, xmax = tremors.merc_long.min(), tremors.merc_long.max()
    ymin, ymax = tremors.merc_lat.min(), tremors.merc_lat.max()

    top = figure(tools=tools, plot_width=600, plot_height=500, title='Tremor locations', x_range=(xmin, xmax), y_range=(ymin, ymax),
               x_axis_type="mercator", y_axis_type="mercator")
    top.add_tile(CARTODBPOSITRON)
    top.circle('merc_long', 'merc_lat', source=unselected_data_source, color='gray', alpha=0.2)
    top.circle('merc_long', 'merc_lat', source=selected_data_source, color='red', alpha=1.0)#, view=view)

    bottom = figure(plot_width=600, plot_height=80, title=None, x_axis_type="datetime")
    bottom.line('date', 'value', source=ColumnDataSource(tremors), color='red', alpha=0.8)

    p = gridplot([[top], [bottom]], toolbar_location='right')
    
    return p

In [3]:
def get_datasets(tremors):
    '''Filter based on widget
    '''
    # start with the whole dataset
    copy = tremors.copy()
    # narrow by selected date from the DateSlider
    selected = copy[copy['datetime']==pd.Timestamp(date_slider.value)]
    unselected = copy[copy['datetime'] < pd.Timestamp(date_slider.value)]

    return {'selected':ColumnDataSource(data=selected), 'unselected':ColumnDataSource(unselected)}

In [4]:
def update_plot():
    src = get_datasets(df)
    selected_data_source.data.update(src['selected'].data)
    unselected_data_source.data.update(src['unselected'].data)

Read tremor data

In [5]:
filename = '../data/tremors.pkl'
tremors = pd.read_pickle(filename)

Keep only 10 per cent of the tremor (to get a smaller dataset)

In [6]:
tremors = tremors.sample(frac=0.1)

Lauch code

In [8]:
date_slider = DateSlider(start=tremors['datetime'].min(), end=tremors['datetime'].max(), step=100, value=tremors['datetime'].min())
date_slider.on_change('value', lambda attr, old, new: update_plot())

slider_box = widgetbox(children=[date_slider], width=600)

datasets_init = get_datasets(tremors) # get bokeh ColumnDataSource
selected_data_source = datasets_init['selected']
unselected_data_source = datasets_init['unselected']
p = make_plot(selected_data_source, unselected_data_source)

layout = layout(children=[[p],[slider_box]], sizing_mode='fixed')

curdoc().add_root(layout)

TypeError: 'Column' object is not callable

In [9]:
from bokeh.plotting import show
show(p)

RuntimeError: Models must be owned by only a single document, WMTSTileSource(id='1001', ...) is already in a doc